# Connection à Kaggle et téléchargement du dossier zip

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arthurkiehl","key":"13acb8548a6c7feacba1276076fa779d"}'}

In [ ]:
# !pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c home-credit-default-risk

 98% 676M/688M [00:06<00:00, 142MB/s]
100% 688M/688M [00:06<00:00, 113MB/s]


## Extraction du dossier zip

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
import zipfile

# Sélection du fichier zip
zip_file_name = "home-credit-default-risk.zip"

# On définit le working directory
working_directory = "/content"


# Dézipper le fichier
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(working_directory)  # Extrayez le contenu du fichier zip dans le working directory


# Exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_col_desc = pd.read_csv('HomeCredit_columns_description.csv', sep=",", encoding='latin1')
df_col_desc.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [ ]:
df_col_desc[['Table','Description']]

,Table,Description
0,application_{train|test}.csv,ID of loan in our sample
1,application_{train|test}.csv,Target variable (1 - client with payment diffi...
2,application_{train|test}.csv,Identification if loan is cash or revolving
3,application_{train|test}.csv,Gender of the client
4,application_{train|test}.csv,Flag if the client owns a car
...,...,...
214,installments_payments.csv,On which installment we observe payment
215,installments_payments.csv,When the installment of previous credit was su...
216,installments_payments.csv,When was the installments of previous credit p...
217,installments_payments.csv,What was the prescribed installment amount of ...


In [ ]:
df_ap = pd.read_csv('application_train.csv')

In [ ]:
df_ap


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
